In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import Aer, execute
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua import QuantumInstance
from qiskit.circuit.library import TwoLocal
from qiskit.aqua.components.optimizers import COBYLA

# Define the graph for the MaxCut problem
w = np.array([[0, 1, 1, 0, 0],
              [1, 0, 1, 1, 0],
              [1, 1, 0, 1, 0],
              [0, 1, 1, 0, 1],
              [0, 0, 0, 1, 0]])

# Create a Quadratic Program for the MaxCut problem
qp = QuadraticProgram()
n = len(w)
for i in range(n):
    qp.binary_var(name=f"x{i}")

objective = 0
for i in range(n):
    for j in range(i):
        if w[i, j] != 0:
            qp.minimize(linear=0, quadratic={(i, j): -w[i, j]})
            objective += w[i, j] * (qp.variables[i] - qp.variables[j]) ** 2

qp.objective.constant = objective

# Convert to QUBO
qp_to_qubo = QuadraticProgramToQubo()
qubo = qp_to_qubo.convert(qp)

# Parameters for QAOA
p_values = [1, 2, 3, 4, 5]
expectation_qiskit = []
expectation_qokit = []

# Qiskit QAOA
backend = Aer.get_backend('qasm_simulator')
optimizer = COBYLA(maxiter=100)

for p in p_values:
    qaoa = QAOA(qubo, optimizer, p=p, quantum_instance=QuantumInstance(backend, shots=1024))
    result = MinimumEigenOptimizer(qaoa).solve(qp)
    expectation_qiskit.append(result.fval)

# QOKit QAOA (Assuming QOKit has similar API to Qiskit)
# from qokit import QAOA, QuantumInstance, Aer
# 
# for p in p_values:
#     qaoa = QAOA(qubo, optimizer, p=p, quantum_instance=QuantumInstance(backend, shots=1024))
#     result = MinimumEigenOptimizer(qaoa).solve(qp)
#     expectation_qokit.append(result.fval)

# Plotting the results
plt.figure(figsize=(10, 6))
plt.plot(p_values, expectation_qiskit, label='Qiskit', marker='o')
plt.plot(p_values, expectation_qokit, label='QOKit', marker='x')
plt.xlabel('Step number p')
plt.ylabel('Expectation value')
plt.title('QAOA for MaxCut: Qiskit vs QOKit')
plt.legend()
plt.grid(True)
plt.show()

ModuleNotFoundError: No module named 'qiskit.optimization'